# Numpy Array Operations:  Axes and Broadcasting 

There is an excellent introduction to `numpy` multi-dimensional arrays on the [scipy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) website.  In this note, we cover two concepts in a little more detail:
* Using the `axis` feature 
* Python broadcasting

We will need both of these for performing many of the numerical operations for the ML class.

As usual, we begin by loading the `numpy` package.

In [46]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [47]:
import numpy as np

## Axis Parameter

Many operations in the `numpy` package can take an optional `axis` parameter to specify which dimensions the operation is to be applied.  This is extremely useful for multi-dimensional data.  To illustrate the `axis` parameter, consider a matrix the `(3,2)` array `X` defined as:

In [15]:
X = np.arange(6).reshape(3,2)
print(X)
print(X[:,1])

[[0 1]
 [2 3]
 [4 5]]
[1 3 5]


An operation like `np.mean` or `np.sum` takes the mean or sum of *all* elements in the array. 

In [9]:
print(np.mean(X))
print(np.sum(X))

2.5
15


To take only the `sum` along each column, we can use the `axis` parameter.

In [10]:
print(np.sum(X,axis=0))

[6 9]


Since `X` has shape `(3,2)`, the output `np.sum(X,axis=0)` is of shape `(2,)`.  Similarly, we can take the `sum` along each row:

In [11]:
print(np.sum(X,axis=1))

[1 5 9]


You can apply this to higher-order arrays:

In [18]:
X = np.arange(24).reshape(2,3,4)  # shape = (2,3,4)
Y1 = np.sum(X,axis=0)             # shape = (3,4)
Y2 = np.sum(X,axis=1)             # shape = (2,4)
print(f'X = {X}')
print('Y1 = ')
print(Y1)
print('Y2 = ')
print(Y2)

X = [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
Y1 = 
[[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]
Y2 = 
[[12 15 18 21]
 [48 51 54 57]]


## Broadcasting

**Broadcasting** is a powerful tool in Python for performing operations on matrices that we will use throughout the ML class.  A good tutorial on broadcasting can be found on the [scipy broadcasting page](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html).  Here we provide some examples.   

### The Broadcasting rule

Arrays with a size of 1 along a particular dimension act as if they had the size of the array with the largest shape along that dimension, by “stretching” the dimensions with size 1.

Specifically, when operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when:
1. they are equal, or
2. one of them is 1

In [58]:
X1 = np.random.randint(0,20,(2,3))
print(X1)
Y2 = np.random.randint(0,20,(1,2))
print(Y2)

[[11 13 11]
 [ 5 19 18]]
[[19  0]]


array([[[ -8,  -6,  -8],
        [ 11,  13,  11]],

       [[-14,   0,  -1],
        [  5,  19,  18]]])

Since X1 and Y2 doesn't meet the broadcasting rule, we need to ad axis to the marices so that they can be computed.

In [59]:
X1[:,None,:]-Y2[:,:,None]

array([[[ -8,  -6,  -8],
        [ 11,  13,  11]],

       [[-14,   0,  -1],
        [  5,  19,  18]]])

### Example 1:  Mean Removal

Suppose that `X` is a data matrix of shape `(n,p)`.  That is, there are `n` data points and `p` features per point.  Often, we have to remove the mean from each feature.  That is, we want to compute the mean for each feature and then remove the mean from each column.  We could do this with a for-loop as:
   
    Xm = np.zeros(p)      # Mean for each feature
    X1_demean = np.zeros((n,p))  # Transformed features with the means removed
    for j in range(p):
       Xm[j] = np.mean(X[:,j])
       for i in range(n):
           X_demean[i,j] = X[i,j] - Xm[j]
           
The code below does this without a for loop using the `axis` parameter and broadcasting.

In [29]:
# Generate some random data
n = 10
p = 5
X = np.arange(n*p).reshape(n,p)
print(f'X = {X}')

# Compute the mean per column using the axis command
Xm = np.mean(X,axis=0)  # This is a p-dim matrix
print(f'Xm = {Xm}')

print(f'Xm[None,:] = {Xm[None,:]}')

# Subtract the mean
X_demean = X - Xm[None,:]
print('X-Xm[None,:]=')
print(f'X_demean = {X_demean}')

X = [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]
 [30 31 32 33 34]
 [35 36 37 38 39]
 [40 41 42 43 44]
 [45 46 47 48 49]]
Xm = [22.5 23.5 24.5 25.5 26.5]
Xm[None,:] = [[22.5 23.5 24.5 25.5 26.5]]
X-Xm[None,:]=
X_demean = [[-22.5 -22.5 -22.5 -22.5 -22.5]
 [-17.5 -17.5 -17.5 -17.5 -17.5]
 [-12.5 -12.5 -12.5 -12.5 -12.5]
 [ -7.5  -7.5  -7.5  -7.5  -7.5]
 [ -2.5  -2.5  -2.5  -2.5  -2.5]
 [  2.5   2.5   2.5   2.5   2.5]
 [  7.5   7.5   7.5   7.5   7.5]
 [ 12.5  12.5  12.5  12.5  12.5]
 [ 17.5  17.5  17.5  17.5  17.5]
 [ 22.5  22.5  22.5  22.5  22.5]]


The command `Xm = np.mean(X,axis=0)` computes the mean of each column which is a `p` dimensional array.  Then, `Xm[None,:]` converts this to a `(1,p)` shape array.  Using Python broadcasting, we can then subtract the `Xm[None,:]` from `X`.

### Example 2:  Standardizing variables

A variant of the above example is to *standardize* the features, where we compute the transform variables,

    Z[i,j] = (X[i,j] - Xm[j])/ Xstd[j]
    
where `Xstd[j]` is the standard deviation per feature.  This can be done as follows:

In [32]:
Xstd = np.std(X,axis=0)
print(f'Xstd = {Xstd}')
Z = (X-Xm[None,:])/Xstd[None,:]
print(f'Z = {Z}')

Xstd = [14.36140662 14.36140662 14.36140662 14.36140662 14.36140662]
Z = [[-1.5666989  -1.5666989  -1.5666989  -1.5666989  -1.5666989 ]
 [-1.21854359 -1.21854359 -1.21854359 -1.21854359 -1.21854359]
 [-0.87038828 -0.87038828 -0.87038828 -0.87038828 -0.87038828]
 [-0.52223297 -0.52223297 -0.52223297 -0.52223297 -0.52223297]
 [-0.17407766 -0.17407766 -0.17407766 -0.17407766 -0.17407766]
 [ 0.17407766  0.17407766  0.17407766  0.17407766  0.17407766]
 [ 0.52223297  0.52223297  0.52223297  0.52223297  0.52223297]
 [ 0.87038828  0.87038828  0.87038828  0.87038828  0.87038828]
 [ 1.21854359  1.21854359  1.21854359  1.21854359  1.21854359]
 [ 1.5666989   1.5666989   1.5666989   1.5666989   1.5666989 ]]


### Example 3:  Distances

Here is a more complicated example.  Suppose we have a data matrix `X` of shape `(nx,p)` and a second set of points, `Y` of shape `(ny,p)`. For each `i` and `j`, we want to compute the distances, 

     d[i,j] = np.sum((X[i,:] - Y[j,:])**2)
     
This represents the distances between the vectors `X[i,:]` and `Y[j,:]`.  This sort of computation is used for clustering and nearest neighbors.  We can do this without a for loop as follows

In [43]:
# Some random data
nx = 4
ny = 3
p = 2
X = np.random.randint(0,20,(nx,p))
Y = np.random.randint(0,5,(ny,p))
print(f'X={X}')
print(f'Y={Y}')
print(f'X[:,None,:] = {X[:,None,:]}')
print(f'X[None,:,:] = {X[None,:,:]}')
print(f'Y[None,:,:] = {Y[None,:,:]}')
# Computing the distances in two lines.  No for loop!
DXY = X[:,None,:]-Y[None,:,:]
d = np.sum(DXY**2,axis=2)
print('DXY=')
print(DXY)

X=[[ 9 19]
 [ 2 10]
 [ 4  7]
 [ 0  9]]
Y=[[3 0]
 [1 2]
 [4 0]]
X[:,None,:] = [[[ 9 19]]

 [[ 2 10]]

 [[ 4  7]]

 [[ 0  9]]]
X[None,:,:] = [[[ 9 19]
  [ 2 10]
  [ 4  7]
  [ 0  9]]]
Y[None,:,:] = [[[3 0]
  [1 2]
  [4 0]]]
DXY=
[[[ 6 19]
  [ 8 17]
  [ 5 19]]

 [[-1 10]
  [ 1  8]
  [-2 10]]

 [[ 1  7]
  [ 3  5]
  [ 0  7]]

 [[-3  9]
  [-1  7]
  [-4  9]]]


How does this work? First, we use `None` keyword to reshape the matrices `X` and `Y` to compatible sizes

     X[:,None,:]    # Shape nx,  1, p
     Y[None,:,:]    # Shape 1,  ny, p
     
The two matrices can be subtracted so that

     DXY[i,j,k]  = X[i,k] - Y[j,k]
     
Then, `d[i,j] = sum_k (X[i,k] - Y[j,k])**2`, which is the norm squared of the vector differences.

### Example 4:  Outer product

The *outer product* of vectors `x` and `y` is the matrix `Z[i,j] = x[i]y[j]`.  This can be performed in one line as follows

In [64]:
# Some random data
nx = 10
ny = 2
X = np.random.rand(nx)
Y = np.random.rand(ny)
print(f'X={X}')
print(f'Y={Y}')
# Compute the outer product in one line
Z = X[:,None]*Y[None,:]
Z

X=[0.47992357 0.2659027  0.14946256 0.51086325 0.92344478 0.9529356
 0.9572224  0.77761824 0.40699581 0.61992902]
Y=[0.25226595 0.33025324]


array([[0.12106838, 0.15849631],
       [0.0670782 , 0.08781523],
       [0.03770432, 0.04936049],
       [0.12887341, 0.16871424],
       [0.23295368, 0.30497063],
       [0.24039321, 0.31471007],
       [0.24147462, 0.3161258 ],
       [0.19616661, 0.25681094],
       [0.10267119, 0.13441168],
       [0.15638699, 0.20473357]])

Here:

     x[:,None] # Has shape (nx,  1)
     y[None,:] # Has shape ( 1, ny)
     
So, with python broadcasting:

     Z = x[:,None]*y[None,:] # has shape (nx,  ny)


## Exercise

**Exercise 1:**  Given a matrix `X`, compute the matrix `Y`, where the rows of `X` are normaized to one.  That is:

     Y[i,j] = X[i,j] / sum_j X[i,j]   

In [77]:
X = np.random.rand(4,3)
print(X)
# Caculate the the sum of each row
X_sum_j = np.sum(X, axis=1)
print(f'X_sum_j = {X_sum_j}')
# The shape of X is (4,3) and the shape of Y should be (4,3), so we need the shape of X_sum_j to be (4,1)
# And new axis at the end of the dimension
Y = X/X_sum_j[:,None]

[[0.67459292 0.26187857 0.0912715 ]
 [0.27043251 0.9976622  0.36720831]
 [0.37976151 0.35713238 0.79053506]
 [0.21885813 0.07215531 0.35665326]]
X_sum_j = [1.027743   1.63530302 1.52742895 0.64766671]


**Exercise 2:** Diagonal multiplication.  Given a matrix `X` and a vector `d`, compute `Y = diag(d)*X`.

In [85]:
X = np.random.rand(5,3)
d = np.random.rand(5)
d_diag = d *np.eye(5)
print(d_diag)
Y = d_diag[:,:,None]*X[:,None,:]
Y

[[0.06949304 0.         0.         0.         0.        ]
 [0.         0.83265415 0.         0.         0.        ]
 [0.         0.         0.48629284 0.         0.        ]
 [0.         0.         0.         0.59811337 0.        ]
 [0.         0.         0.         0.         0.88559345]]


array([[[0.06379967, 0.01245082, 0.06448699],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.13865168, 0.79475176, 0.70934603],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.31543407, 0.46500495, 0.17490456],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.5002267 , 0.40818013, 0.31766957],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.   